### Importación de librerías

In [1]:
#Se importan las librerias a utilizar

import numpy as np
import pandas as pd
import os

import librosa
import librosa.display

from scipy.stats import skew, kurtosis

----

### Lectura de los datasets

In [2]:
urbansound8k_path = "../../data/raw/UrbanSound8K/"

# Primer dataset - Carpeta con los audios
urbansound8k_audio_path = "audio/"

# Segundo dataset - Archivo CSV con metadatos sobre los audios 
urbansound8k_metadata_path_file = "metadata/UrbanSound8K.csv"

In [3]:
dataset_urbansound8k_df = pd.read_csv(urbansound8k_path + urbansound8k_metadata_path_file, sep=",")
dataset_urbansound8k_df.head(2)

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing


---
### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos 
4. Formateo definitivo


----

#### Eliminación de columnas 

In [4]:
# Se establece qué columnas se eliminan
columnas_eliminar_lista = ['fsID', 'start', 'end', 'class']

# Se ejecuta la operación
dataset_urbansound8k_df.drop(columnas_eliminar_lista, inplace=True, axis=1) 

In [5]:
print("Vista del dataset de datos:")
display(dataset_urbansound8k_df.head(1))

Vista del dataset de datos:


,slice_file_name,salience,fold,classID
0,100032-3-0-0.wav,1,5,3


In [6]:
print("Columnas del dataset:")
print(dataset_urbansound8k_df.columns)
print("---")

print(f"Cantidad de columnas: {dataset_urbansound8k_df.shape[1]}")
print(f"Cantidad de filas: {dataset_urbansound8k_df.shape[0]}")

Columnas del dataset:
Index(['slice_file_name', 'salience', 'fold', 'classID'], dtype='object')
---
Cantidad de columnas: 4
Cantidad de filas: 8732


#### Construcción de atributos MFCC10

In [7]:
def extract_mfcc(file_path, n_mfcc=13):
    signal, sr = librosa.load(file_path, sr=44100, mono=True)  # Cargar el archivo de audio
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_mfcc)  # Extraer MFCC

    # Calcular estadísticas de los MFCC
    mfcc_mean = np.mean(mfccs, axis=1)
    mfcc_median = np.median(mfccs, axis=1)
    mfcc_min = np.min(mfccs, axis=1)
    mfcc_max = np.max(mfccs, axis=1)
    mfcc_var = np.var(mfccs, axis=1)
    mfcc_skew = skew(mfccs, axis=1)
    mfcc_kurt = kurtosis(mfccs, axis=1)

    # Ajustar el tamaño de la ventana si el número de frames es pequeño
    num_frames = mfccs.shape[1]
    delta_width = min(3, num_frames - 1)  # Ajustar el ancho para evitar el error

    # Calcular derivadas (primera y segunda)
    delta_mfcc = librosa.feature.delta(mfccs, width=delta_width)
    delta2_mfcc = librosa.feature.delta(mfccs, order=2, width=delta_width)

    # Estadísticas de las derivadas
    delta_mfcc_mean = np.mean(delta_mfcc, axis=1)
    delta_mfcc_var = np.var(delta_mfcc, axis=1)
    delta2_mfcc_mean = np.mean(delta2_mfcc, axis=1)
    delta2_mfcc_var = np.var(delta2_mfcc, axis=1)

    # Combinar todas las características en un solo vector
    features = np.hstack([mfcc_mean, mfcc_median, mfcc_min, mfcc_max, mfcc_var, mfcc_skew, mfcc_kurt,
                          delta_mfcc_mean, delta_mfcc_var, delta2_mfcc_mean, delta2_mfcc_var])

    return features  

In [8]:
n_mfcc = 10 

# Crear columnas para cada una de las características
mfcc_columns = [f'mfcc_mean_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_median_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_min_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_max_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_var_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_skew_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_kurt_{i+1}' for i in range(n_mfcc)] + \
               [f'delta_mfcc_mean_{i+1}' for i in range(n_mfcc)] + \
               [f'delta_mfcc_var_{i+1}' for i in range(n_mfcc)] + \
               [f'delta2_mfcc_mean_{i+1}' for i in range(n_mfcc)] + \
               [f'delta2_mfcc_var_{i+1}' for i in range(n_mfcc)]

mfcc_features = []
# Extraer MFCCs para todos los archivos
for index, row in dataset_urbansound8k_df.iterrows():
    path_total_file = f"{urbansound8k_path}/audio/fold{row['fold']}/{row['slice_file_name']}"
    mfcc = extract_mfcc(path_total_file, n_mfcc)
    mfcc_features.append(mfcc)

# Convertir la lista de MFCCs en un DataFrame y unirlo al DataFrame original
mfcc_df = pd.DataFrame(mfcc_features, columns=mfcc_columns)
dataset_urbansound8k_mfcc10_df = pd.concat([dataset_urbansound8k_df, mfcc_df], axis=1)

In [9]:
dataset_urbansound8k_mfcc10_df.tail(3)

,slice_file_name,salience,fold,classID,mfcc_mean_1,mfcc_mean_2,mfcc_mean_3,mfcc_mean_4,mfcc_mean_5,mfcc_mean_6,...,delta2_mfcc_var_1,delta2_mfcc_var_2,delta2_mfcc_var_3,delta2_mfcc_var_4,delta2_mfcc_var_5,delta2_mfcc_var_6,delta2_mfcc_var_7,delta2_mfcc_var_8,delta2_mfcc_var_9,delta2_mfcc_var_10
8729,99812-1-4-0.wav,2,7,1,-362.703217,165.081787,-60.780693,30.996469,-9.655406,23.180807,...,71.702911,37.743973,30.453892,33.198425,26.695854,29.252632,31.280785,28.162518,28.256092,26.317396
8730,99812-1-5-0.wav,2,7,1,-404.628113,178.504730,-63.615627,25.944529,-10.428956,12.489921,...,52.384373,30.892416,29.776909,25.414547,29.228724,25.150442,28.594847,30.689117,21.841654,24.348642
8731,99812-1-6-0.wav,2,7,1,-369.637054,147.904556,-63.150425,47.780521,-9.488717,33.391727,...,58.536484,40.776539,29.844807,29.965380,27.273800,24.325298,35.004864,29.814854,40.410534,27.095753


In [10]:
print("Columnas del dataset:")
pd.set_option('display.max_columns', None)
print(dataset_urbansound8k_mfcc10_df.columns)
print("---")

print(f"Cantidad de columnas: {dataset_urbansound8k_mfcc10_df.shape[1]}")
print(f"Cantidad de filas: {dataset_urbansound8k_mfcc10_df.shape[0]}")

Columnas del dataset:
Index(['slice_file_name', 'salience', 'fold', 'classID', 'mfcc_mean_1',
       'mfcc_mean_2', 'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5',
       'mfcc_mean_6',
       ...
       'delta2_mfcc_var_1', 'delta2_mfcc_var_2', 'delta2_mfcc_var_3',
       'delta2_mfcc_var_4', 'delta2_mfcc_var_5', 'delta2_mfcc_var_6',
       'delta2_mfcc_var_7', 'delta2_mfcc_var_8', 'delta2_mfcc_var_9',
       'delta2_mfcc_var_10'],
      dtype='object', length=114)
---
Cantidad de columnas: 114
Cantidad de filas: 8732


#### Construcción de atributos MFCC13

In [11]:
n_mfcc = 13 

# Crear columnas para cada una de las características
mfcc_columns = [f'mfcc_mean_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_median_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_min_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_max_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_var_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_skew_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_kurt_{i+1}' for i in range(n_mfcc)] + \
               [f'delta_mfcc_mean_{i+1}' for i in range(n_mfcc)] + \
               [f'delta_mfcc_var_{i+1}' for i in range(n_mfcc)] + \
               [f'delta2_mfcc_mean_{i+1}' for i in range(n_mfcc)] + \
               [f'delta2_mfcc_var_{i+1}' for i in range(n_mfcc)]

mfcc_features = []
# Extraer MFCCs para todos los archivos
for index, row in dataset_urbansound8k_df.iterrows():
    path_total_file = f"{urbansound8k_path}/audio/fold{row['fold']}/{row['slice_file_name']}"
    mfcc = extract_mfcc(path_total_file, n_mfcc)
    mfcc_features.append(mfcc)

# Convertir la lista de MFCCs en un DataFrame y unirlo al DataFrame original
mfcc_df = pd.DataFrame(mfcc_features, columns=mfcc_columns)
dataset_urbansound8k_mfcc13_df = pd.concat([dataset_urbansound8k_df, mfcc_df], axis=1)

In [12]:
dataset_urbansound8k_mfcc13_df.tail(3)

,slice_file_name,salience,fold,classID,mfcc_mean_1,mfcc_mean_2,mfcc_mean_3,mfcc_mean_4,mfcc_mean_5,mfcc_mean_6,mfcc_mean_7,mfcc_mean_8,mfcc_mean_9,mfcc_mean_10,mfcc_mean_11,mfcc_mean_12,mfcc_mean_13,mfcc_median_1,mfcc_median_2,mfcc_median_3,mfcc_median_4,mfcc_median_5,mfcc_median_6,mfcc_median_7,mfcc_median_8,mfcc_median_9,mfcc_median_10,mfcc_median_11,mfcc_median_12,mfcc_median_13,mfcc_min_1,mfcc_min_2,mfcc_min_3,mfcc_min_4,mfcc_min_5,mfcc_min_6,mfcc_min_7,mfcc_min_8,mfcc_min_9,mfcc_min_10,mfcc_min_11,mfcc_min_12,mfcc_min_13,mfcc_max_1,mfcc_max_2,mfcc_max_3,mfcc_max_4,mfcc_max_5,mfcc_max_6,mfcc_max_7,mfcc_max_8,mfcc_max_9,mfcc_max_10,mfcc_max_11,mfcc_max_12,mfcc_max_13,mfcc_var_1,mfcc_var_2,mfcc_var_3,mfcc_var_4,mfcc_var_5,mfcc_var_6,mfcc_var_7,mfcc_var_8,mfcc_var_9,mfcc_var_10,mfcc_var_11,mfcc_var_12,mfcc_var_13,mfcc_skew_1,mfcc_skew_2,mfcc_skew_3,mfcc_skew_4,mfcc_skew_5,mfcc_skew_6,mfcc_skew_7,mfcc_skew_8,mfcc_skew_9,mfcc_skew_10,mfcc_skew_11,mfcc_skew_12,mfcc_skew_13,mfcc_kurt_1,mfcc_kurt_2,mfcc_kurt_3,mfcc_kurt_4,mfcc_kurt_5,mfcc_kurt_6,mfcc_kurt_7,mfcc_kurt_8,mfcc_kurt_9,mfcc_kurt_10,mfcc_kurt_11,mfcc_kurt_12,mfcc_kurt_13,delta_mfcc_mean_1,delta_mfcc_mean_2,delta_mfcc_mean_3,delta_mfcc_mean_4,delta_mfcc_mean_5,delta_mfcc_mean_6,delta_mfcc_mean_7,delta_mfcc_mean_8,delta_mfcc_mean_9,delta_mfcc_mean_10,delta_mfcc_mean_11,delta_mfcc_mean_12,delta_mfcc_mean_13,delta_mfcc_var_1,delta_mfcc_var_2,delta_mfcc_var_3,delta_mfcc_var_4,delta_mfcc_var_5,delta_mfcc_var_6,delta_mfcc_var_7,delta_mfcc_var_8,delta_mfcc_var_9,delta_mfcc_var_10,delta_mfcc_var_11,delta_mfcc_var_12,delta_mfcc_var_13,delta2_mfcc_mean_1,delta2_mfcc_mean_2,delta2_mfcc_mean_3,delta2_mfcc_mean_4,delta2_mfcc_mean_5,delta2_mfcc_mean_6,delta2_mfcc_mean_7,delta2_mfcc_mean_8,delta2_mfcc_mean_9,delta2_mfcc_mean_10,delta2_mfcc_mean_11,delta2_mfcc_mean_12,delta2_mfcc_mean_13,delta2_mfcc_var_1,delta2_mfcc_var_2,delta2_mfcc_var_3,delta2_mfcc_var_4,delta2_mfcc_var_5,delta2_mfcc_var_6,delta2_mfcc_var_7,delta2_mfcc_var_8,delta2_mfcc_var_9,delta2_mfcc_var_10,delta2_mfcc_var_11,delta2_mfcc_var_12,delta2_mfcc_var_13
8729,99812-1-4-0.wav,2,7,1,-362.703217,165.081787,-60.780693,30.996469,-9.655406,23.180807,-18.668148,27.321251,-7.465636,7.173614,-1.006558,-3.445722,2.150398,-374.618408,172.437561,-58.439495,26.898571,-8.560820,21.448959,-16.386528,26.489162,-6.225329,7.376889,0.166195,-2.942360,1.409455,-430.470337,87.967751,-105.449814,5.101768,-35.521935,3.919481,-58.861862,6.278496,-32.665653,-6.544235,-22.682743,-25.178654,-10.114086,-218.474197,197.326645,-19.847599,84.176079,10.262193,44.737865,1.287581,50.585827,4.794589,23.797684,20.188076,12.089788,20.960880,2538.087402,592.256470,229.116119,306.218903,79.402695,78.482834,120.956810,64.004143,46.478008,21.676336,66.413918,30.873665,25.775778,1.456478,-1.764216,-0.883883,1.380343,-0.705838,0.361428,-1.752563,0.432333,-1.096469,-0.037268,-0.539259,-0.525329,0.512520,1.590599,2.487436,0.616282,1.322091,0.331387,-0.594940,3.029640,0.286762,1.357313,0.092870,0.003850,0.858595,0.403210,-0.193360,0.130044,-0.110927,0.086948,-0.064086,0.026170,-0.086512,0.004189,-0.053388,0.034958,0.028094,0.039136,0.024142,81.227150,25.196516,13.822035,15.564625,8.860290,10.084868,13.570687,9.952473,8.147461,6.363957,8.087103,9.029004,8.355199,-0.094859,0.030502,0.063619,0.014380,-0.004367,-0.037876,-0.010383,-0.028187,-0.028025,-0.016724,0.004343,0.001841,-0.020912,71.702911,37.743973,30.453892,33.198425,26.695854,29.252632,31.280785,28.162518,28.256092,26.317396,27.488977,35.974876,33.799191
8730,99812-1-5-0.wav,2,7,1,-404.628113,178.504730,-63.615627,25.944529,-10.428956,12.489921,-24.788843,35.468410,-3.097389,0.915603,3.893663,4.589596,2.739587,-402.621918,178.015747,-64.721230,26.687176,-10.810170,12.649281,-25.849751,37.497284,-3.425486,1.672693,3.261973,5.067974,1.333479,-460.158020,156.088715,-78.231506,4.116554,-26.282526,-0.626234,-48.766792,10.349158,-17.463551,-21.145693,-15.546354,-12.739191,-17.440796,-353.516541,202.885071,-40.3456

In [13]:
print("Columnas del dataset:")
pd.set_option('display.max_columns', None)
print(dataset_urbansound8k_mfcc13_df.columns)
print("---")

print(f"Cantidad de columnas: {dataset_urbansound8k_mfcc13_df.shape[1]}")
print(f"Cantidad de filas: {dataset_urbansound8k_mfcc13_df.shape[0]}")

Columnas del dataset:
Index(['slice_file_name', 'salience', 'fold', 'classID', 'mfcc_mean_1',
       'mfcc_mean_2', 'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5',
       'mfcc_mean_6',
       ...
       'delta2_mfcc_var_4', 'delta2_mfcc_var_5', 'delta2_mfcc_var_6',
       'delta2_mfcc_var_7', 'delta2_mfcc_var_8', 'delta2_mfcc_var_9',
       'delta2_mfcc_var_10', 'delta2_mfcc_var_11', 'delta2_mfcc_var_12',
       'delta2_mfcc_var_13'],
      dtype='object', length=147)
---
Cantidad de columnas: 147
Cantidad de filas: 8732


#### Construcción de atributos MFCC15

In [14]:
n_mfcc = 15 

# Crear columnas para cada una de las características
mfcc_columns = [f'mfcc_mean_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_median_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_min_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_max_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_var_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_skew_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_kurt_{i+1}' for i in range(n_mfcc)] + \
               [f'delta_mfcc_mean_{i+1}' for i in range(n_mfcc)] + \
               [f'delta_mfcc_var_{i+1}' for i in range(n_mfcc)] + \
               [f'delta2_mfcc_mean_{i+1}' for i in range(n_mfcc)] + \
               [f'delta2_mfcc_var_{i+1}' for i in range(n_mfcc)]

mfcc_features = []
# Extraer MFCCs para todos los archivos
for index, row in dataset_urbansound8k_df.iterrows():
    path_total_file = f"{urbansound8k_path}/audio/fold{row['fold']}/{row['slice_file_name']}"
    mfcc = extract_mfcc(path_total_file, n_mfcc)
    mfcc_features.append(mfcc)

# Convertir la lista de MFCCs en un DataFrame y unirlo al DataFrame original
mfcc_df = pd.DataFrame(mfcc_features, columns=mfcc_columns)
dataset_urbansound8k_mfcc15_df = pd.concat([dataset_urbansound8k_df, mfcc_df], axis=1)

In [15]:
dataset_urbansound8k_mfcc15_df.tail(3)

,slice_file_name,salience,fold,classID,mfcc_mean_1,mfcc_mean_2,mfcc_mean_3,mfcc_mean_4,mfcc_mean_5,mfcc_mean_6,mfcc_mean_7,mfcc_mean_8,mfcc_mean_9,mfcc_mean_10,mfcc_mean_11,mfcc_mean_12,mfcc_mean_13,mfcc_mean_14,mfcc_mean_15,mfcc_median_1,mfcc_median_2,mfcc_median_3,mfcc_median_4,mfcc_median_5,mfcc_median_6,mfcc_median_7,mfcc_median_8,mfcc_median_9,mfcc_median_10,mfcc_median_11,mfcc_median_12,mfcc_median_13,mfcc_median_14,mfcc_median_15,mfcc_min_1,mfcc_min_2,mfcc_min_3,mfcc_min_4,mfcc_min_5,mfcc_min_6,mfcc_min_7,mfcc_min_8,mfcc_min_9,mfcc_min_10,mfcc_min_11,mfcc_min_12,mfcc_min_13,mfcc_min_14,mfcc_min_15,mfcc_max_1,mfcc_max_2,mfcc_max_3,mfcc_max_4,mfcc_max_5,mfcc_max_6,mfcc_max_7,mfcc_max_8,mfcc_max_9,mfcc_max_10,mfcc_max_11,mfcc_max_12,mfcc_max_13,mfcc_max_14,mfcc_max_15,mfcc_var_1,mfcc_var_2,mfcc_var_3,mfcc_var_4,mfcc_var_5,mfcc_var_6,mfcc_var_7,mfcc_var_8,mfcc_var_9,mfcc_var_10,mfcc_var_11,mfcc_var_12,mfcc_var_13,mfcc_var_14,mfcc_var_15,mfcc_skew_1,mfcc_skew_2,mfcc_skew_3,mfcc_skew_4,mfcc_skew_5,mfcc_skew_6,mfcc_skew_7,mfcc_skew_8,mfcc_skew_9,mfcc_skew_10,mfcc_skew_11,mfcc_skew_12,mfcc_skew_13,mfcc_skew_14,mfcc_skew_15,mfcc_kurt_1,mfcc_kurt_2,mfcc_kurt_3,mfcc_kurt_4,mfcc_kurt_5,mfcc_kurt_6,mfcc_kurt_7,mfcc_kurt_8,mfcc_kurt_9,mfcc_kurt_10,mfcc_kurt_11,mfcc_kurt_12,mfcc_kurt_13,mfcc_kurt_14,mfcc_kurt_15,delta_mfcc_mean_1,delta_mfcc_mean_2,delta_mfcc_mean_3,delta_mfcc_mean_4,delta_mfcc_mean_5,delta_mfcc_mean_6,delta_mfcc_mean_7,delta_mfcc_mean_8,delta_mfcc_mean_9,delta_mfcc_mean_10,delta_mfcc_mean_11,delta_mfcc_mean_12,delta_mfcc_mean_13,delta_mfcc_mean_14,delta_mfcc_mean_15,delta_mfcc_var_1,delta_mfcc_var_2,delta_mfcc_var_3,delta_mfcc_var_4,delta_mfcc_var_5,delta_mfcc_var_6,delta_mfcc_var_7,delta_mfcc_var_8,delta_mfcc_var_9,delta_mfcc_var_10,delta_mfcc_var_11,delta_mfcc_var_12,delta_mfcc_var_13,delta_mfcc_var_14,delta_mfcc_var_15,delta2_mfcc_mean_1,delta2_mfcc_mean_2,delta2_mfcc_mean_3,delta2_mfcc_mean_4,delta2_mfcc_mean_5,delta2_mfcc_mean_6,delta2_mfcc_mean_7,delta2_mfcc_mean_8,delta2_mfcc_mean_9,delta2_mfcc_mean_10,delta2_mfcc_mean_11,delta2_mfcc_mean_12,delta2_mfcc_mean_13,delta2_mfcc_mean_14,delta2_mfcc_mean_15,delta2_mfcc_var_1,delta2_mfcc_var_2,delta2_mfcc_var_3,delta2_mfcc_var_4,delta2_mfcc_var_5,delta2_mfcc_var_6,delta2_mfcc_var_7,delta2_mfcc_var_8,delta2_mfcc_var_9,delta2_mfcc_var_10,delta2_mfcc_var_11,delta2_mfcc_var_12,delta2_mfcc_var_13,delta2_mfcc_var_14,delta2_mfcc_var_15
8729,99812-1-4-0.wav,2,7,1,-362.703217,165.081787,-60.780693,30.996469,-9.655406,23.180807,-18.668148,27.321251,-7.465636,7.173614,-1.006558,-3.445722,2.150398,-6.810231,-5.280443,-374.618408,172.437561,-58.439495,26.898571,-8.560820,21.448959,-16.386528,26.489162,-6.225329,7.376889,0.166195,-2.942360,1.409455,-6.530595,-4.753449,-430.470337,87.967751,-105.449814,5.101768,-35.521935,3.919481,-58.861862,6.278496,-32.665653,-6.544235,-22.682743,-25.178654,-10.114086,-20.279682,-19.501732,-218.474197,197.326645,-19.847599,84.176079,10.262193,44.737865,1.287581,50.585827,4.794589,23.797684,20.188076,12.089788,20.960880,7.282120,11.316782,2538.087402,592.256470,229.116119,306.218903,79.402695,78.482834,120.956810,64.004143,46.478008,21.676336,66.413918,30.873665,25.775778,35.274342,36.300232,1.456478,-1.764216,-0.883883,1.380343,-0.705838,0.361428,-1.752563,0.432333,-1.096469,-0.037268,-0.539259,-0.525329,0.512520,0.046873,-0.122355,1.590599,2.487436,0.616282,1.322091,0.331387,-0.594940,3.029640,0.286762,1.357313,0.092870,0.003850,0.858595,0.403210,-0.640145,-0.391838,-0.193360,0.130044,-0.110927,0.086948,-0.064086,0.026170,-0.086512,0.004189,-0.053388,0.034958,0.028094,0.039136,0.024142,0.041908,0.080465,81.227150,25.196516,13.822035,15.564625,8.860290,10.084868,13.570687,9.952473,8.147461,6.363957,8.087103,9.029004,8.355199,8.051218,7.322807,-0.094859,0.030502,0.063619,0.014380,-0.004367,-0.037876,-0.010383,-0.028187,-0.028025,-0.016724,0.004343,0.001841,-0.020912,0.023788,0.004113,71.702911,37.743973,30.453892,33.198425,26.695854,29.252632,31.280

In [16]:
print("Columnas del dataset:")
pd.set_option('display.max_columns', None)
print(dataset_urbansound8k_mfcc15_df.columns)
print("---")

print(f"Cantidad de columnas: {dataset_urbansound8k_mfcc15_df.shape[1]}")
print(f"Cantidad de filas: {dataset_urbansound8k_mfcc15_df.shape[0]}")

Columnas del dataset:
Index(['slice_file_name', 'salience', 'fold', 'classID', 'mfcc_mean_1',
       'mfcc_mean_2', 'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5',
       'mfcc_mean_6',
       ...
       'delta2_mfcc_var_6', 'delta2_mfcc_var_7', 'delta2_mfcc_var_8',
       'delta2_mfcc_var_9', 'delta2_mfcc_var_10', 'delta2_mfcc_var_11',
       'delta2_mfcc_var_12', 'delta2_mfcc_var_13', 'delta2_mfcc_var_14',
       'delta2_mfcc_var_15'],
      dtype='object', length=169)
---
Cantidad de columnas: 169
Cantidad de filas: 8732


#### Construcción de atributos MFCC20

In [17]:
n_mfcc = 20 

# Crear columnas para cada una de las características
mfcc_columns = [f'mfcc_mean_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_median_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_min_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_max_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_var_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_skew_{i+1}' for i in range(n_mfcc)] + \
               [f'mfcc_kurt_{i+1}' for i in range(n_mfcc)] + \
               [f'delta_mfcc_mean_{i+1}' for i in range(n_mfcc)] + \
               [f'delta_mfcc_var_{i+1}' for i in range(n_mfcc)] + \
               [f'delta2_mfcc_mean_{i+1}' for i in range(n_mfcc)] + \
               [f'delta2_mfcc_var_{i+1}' for i in range(n_mfcc)]

mfcc_features = []
# Extraer MFCCs para todos los archivos
for index, row in dataset_urbansound8k_df.iterrows():
    path_total_file = f"{urbansound8k_path}/audio/fold{row['fold']}/{row['slice_file_name']}"
    mfcc = extract_mfcc(path_total_file, n_mfcc)
    mfcc_features.append(mfcc)

# Convertir la lista de MFCCs en un DataFrame y unirlo al DataFrame original
mfcc_df = pd.DataFrame(mfcc_features, columns=mfcc_columns)
dataset_urbansound8k_mfcc20_df = pd.concat([dataset_urbansound8k_df, mfcc_df], axis=1)

In [18]:
dataset_urbansound8k_mfcc20_df.tail(3)

,slice_file_name,salience,fold,classID,mfcc_mean_1,mfcc_mean_2,mfcc_mean_3,mfcc_mean_4,mfcc_mean_5,mfcc_mean_6,mfcc_mean_7,mfcc_mean_8,mfcc_mean_9,mfcc_mean_10,mfcc_mean_11,mfcc_mean_12,mfcc_mean_13,mfcc_mean_14,mfcc_mean_15,mfcc_mean_16,mfcc_mean_17,mfcc_mean_18,mfcc_mean_19,mfcc_mean_20,mfcc_median_1,mfcc_median_2,mfcc_median_3,mfcc_median_4,mfcc_median_5,mfcc_median_6,mfcc_median_7,mfcc_median_8,mfcc_median_9,mfcc_median_10,mfcc_median_11,mfcc_median_12,mfcc_median_13,mfcc_median_14,mfcc_median_15,mfcc_median_16,mfcc_median_17,mfcc_median_18,mfcc_median_19,mfcc_median_20,mfcc_min_1,mfcc_min_2,mfcc_min_3,mfcc_min_4,mfcc_min_5,mfcc_min_6,mfcc_min_7,mfcc_min_8,mfcc_min_9,mfcc_min_10,mfcc_min_11,mfcc_min_12,mfcc_min_13,mfcc_min_14,mfcc_min_15,mfcc_min_16,mfcc_min_17,mfcc_min_18,mfcc_min_19,mfcc_min_20,mfcc_max_1,mfcc_max_2,mfcc_max_3,mfcc_max_4,mfcc_max_5,mfcc_max_6,mfcc_max_7,mfcc_max_8,mfcc_max_9,mfcc_max_10,mfcc_max_11,mfcc_max_12,mfcc_max_13,mfcc_max_14,mfcc_max_15,mfcc_max_16,mfcc_max_17,mfcc_max_18,mfcc_max_19,mfcc_max_20,mfcc_var_1,mfcc_var_2,mfcc_var_3,mfcc_var_4,mfcc_var_5,mfcc_var_6,mfcc_var_7,mfcc_var_8,mfcc_var_9,mfcc_var_10,mfcc_var_11,mfcc_var_12,mfcc_var_13,mfcc_var_14,mfcc_var_15,mfcc_var_16,mfcc_var_17,mfcc_var_18,mfcc_var_19,mfcc_var_20,mfcc_skew_1,mfcc_skew_2,mfcc_skew_3,mfcc_skew_4,mfcc_skew_5,mfcc_skew_6,mfcc_skew_7,mfcc_skew_8,mfcc_skew_9,mfcc_skew_10,mfcc_skew_11,mfcc_skew_12,mfcc_skew_13,mfcc_skew_14,mfcc_skew_15,mfcc_skew_16,mfcc_skew_17,mfcc_skew_18,mfcc_skew_19,mfcc_skew_20,mfcc_kurt_1,mfcc_kurt_2,mfcc_kurt_3,mfcc_kurt_4,mfcc_kurt_5,mfcc_kurt_6,mfcc_kurt_7,mfcc_kurt_8,mfcc_kurt_9,mfcc_kurt_10,mfcc_kurt_11,mfcc_kurt_12,mfcc_kurt_13,mfcc_kurt_14,mfcc_kurt_15,mfcc_kurt_16,mfcc_kurt_17,mfcc_kurt_18,mfcc_kurt_19,mfcc_kurt_20,delta_mfcc_mean_1,delta_mfcc_mean_2,delta_mfcc_mean_3,delta_mfcc_mean_4,delta_mfcc_mean_5,delta_mfcc_mean_6,delta_mfcc_mean_7,delta_mfcc_mean_8,delta_mfcc_mean_9,delta_mfcc_mean_10,delta_mfcc_mean_11,delta_mfcc_mean_12,delta_mfcc_mean_13,delta_mfcc_mean_14,delta_mfcc_mean_15,delta_mfcc_mean_16,delta_mfcc_mean_17,delta_mfcc_mean_18,delta_mfcc_mean_19,delta_mfcc_mean_20,delta_mfcc_var_1,delta_mfcc_var_2,delta_mfcc_var_3,delta_mfcc_var_4,delta_mfcc_var_5,delta_mfcc_var_6,delta_mfcc_var_7,delta_mfcc_var_8,delta_mfcc_var_9,delta_mfcc_var_10,delta_mfcc_var_11,delta_mfcc_var_12,delta_mfcc_var_13,delta_mfcc_var_14,delta_mfcc_var_15,delta_mfcc_var_16,delta_mfcc_var_17,delta_mfcc_var_18,delta_mfcc_var_19,delta_mfcc_var_20,delta2_mfcc_mean_1,delta2_mfcc_mean_2,delta2_mfcc_mean_3,delta2_mfcc_mean_4,delta2_mfcc_mean_5,delta2_mfcc_mean_6,delta2_mfcc_mean_7,delta2_mfcc_mean_8,delta2_mfcc_mean_9,delta2_mfcc_mean_10,delta2_mfcc_mean_11,delta2_mfcc_mean_12,delta2_mfcc_mean_13,delta2_mfcc_mean_14,delta2_mfcc_mean_15,delta2_mfcc_mean_16,delta2_mfcc_mean_17,delta2_mfcc_mean_18,delta2_mfcc_mean_19,delta2_mfcc_mean_20,delta2_mfcc_var_1,delta2_mfcc_var_2,delta2_mfcc_var_3,delta2_mfcc_var_4,delta2_mfcc_var_5,delta2_mfcc_var_6,delta2_mfcc_var_7,delta2_mfcc_var_8,delta2_mfcc_var_9,delta2_mfcc_var_10,delta2_mfcc_var_11,delta2_mfcc_var_12,delta2_mfcc_var_13,delta2_mfcc_var_14,delta2_mfcc_var_15,delta2_mfcc_var_16,delta2_mfcc_var_17,delta2_mfcc_var_18,delta2_mfcc_var_19,delta2_mfcc_var_20
8729,99812-1-4-0.wav,2,7,1,-362.703217,165.081787,-60.780693,30.996469,-9.655406,23.180807,-18.668148,27.321251,-7.465636,7.173614,-1.006558,-3.445722,2.150398,-6.810231,-5.280443,-5.075459,6.409055,-2.177091,7.115071,-5.063334,-374.618408,172.437561,-58.439495,26.898571,-8.560820,21.448959,-16.386528,26.489162,-6.225329,7.376889,0.166195,-2.942360,1.409455,-6.530595,-4.753449,-5.141109,5.820332,-1.383223,7.623973,-5.534404,-430.470337,87.967751,-105.449814,5.101768,-35.521935,3.919481,-58.861862,6.278496,-32.665653,-6.544235,-22.682743,-25.178654,-10.114086,-20.279682,-19.501732,-17.293453,-5.424758,-22.827829,-12.464821,-17.119438,-218.474197,197.326645,-19.847599,84.176079,10.262193,44.737865,1.287581,50.585827,4.794589,23.7976

In [19]:
print("Columnas del dataset:")
pd.set_option('display.max_columns', None)
print(dataset_urbansound8k_mfcc20_df.columns)
print("---")

print(f"Cantidad de columnas: {dataset_urbansound8k_mfcc20_df.shape[1]}")
print(f"Cantidad de filas: {dataset_urbansound8k_mfcc20_df.shape[0]}")

Columnas del dataset:
Index(['slice_file_name', 'salience', 'fold', 'classID', 'mfcc_mean_1',
       'mfcc_mean_2', 'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5',
       'mfcc_mean_6',
       ...
       'delta2_mfcc_var_11', 'delta2_mfcc_var_12', 'delta2_mfcc_var_13',
       'delta2_mfcc_var_14', 'delta2_mfcc_var_15', 'delta2_mfcc_var_16',
       'delta2_mfcc_var_17', 'delta2_mfcc_var_18', 'delta2_mfcc_var_19',
       'delta2_mfcc_var_20'],
      dtype='object', length=224)
---
Cantidad de columnas: 224
Cantidad de filas: 8732


#### Mostrar columnas 

In [20]:
contador = 0
for name in dataset_urbansound8k_mfcc13_df.columns:
    contador +=1
    if contador%5==0:
        contador = 0
        print("")        
    print(f"'{name}'", end=", ")

'slice_file_name', 'salience', 'fold', 'classID', 
'mfcc_mean_1', 'mfcc_mean_2', 'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5', 
'mfcc_mean_6', 'mfcc_mean_7', 'mfcc_mean_8', 'mfcc_mean_9', 'mfcc_mean_10', 
'mfcc_mean_11', 'mfcc_mean_12', 'mfcc_mean_13', 'mfcc_median_1', 'mfcc_median_2', 
'mfcc_median_3', 'mfcc_median_4', 'mfcc_median_5', 'mfcc_median_6', 'mfcc_median_7', 
'mfcc_median_8', 'mfcc_median_9', 'mfcc_median_10', 'mfcc_median_11', 'mfcc_median_12', 
'mfcc_median_13', 'mfcc_min_1', 'mfcc_min_2', 'mfcc_min_3', 'mfcc_min_4', 
'mfcc_min_5', 'mfcc_min_6', 'mfcc_min_7', 'mfcc_min_8', 'mfcc_min_9', 
'mfcc_min_10', 'mfcc_min_11', 'mfcc_min_12', 'mfcc_min_13', 'mfcc_max_1', 
'mfcc_max_2', 'mfcc_max_3', 'mfcc_max_4', 'mfcc_max_5', 'mfcc_max_6', 
'mfcc_max_7', 'mfcc_max_8', 'mfcc_max_9', 'mfcc_max_10', 'mfcc_max_11', 
'mfcc_max_12', 'mfcc_max_13', 'mfcc_var_1', 'mfcc_var_2', 'mfcc_var_3', 
'mfcc_var_4', 'mfcc_var_5', 'mfcc_var_6', 'mfcc_var_7', 'mfcc_var_8', 
'mfcc_var_9', 'mfcc_var_10', 

#### Filtrado de filas - Primer Plano

In [21]:
filtered_salience_top_1_df = dataset_urbansound8k_mfcc13_df.loc[dataset_urbansound8k_mfcc13_df['salience'] == 1]


print("Campo salience : Una calificación (subjetiva) de la prominencia del sonido. ")
print("Valores: 1 = primer plano, 2 = fondo. ")
print("Se filtra los audios con calificación de primer plano. \n")

cantidad_filas_filtradas = dataset_urbansound8k_mfcc13_df.shape[0] - filtered_salience_top_1_df.shape[0]
print(f"Cantidad filas afectadas por filtro: {cantidad_filas_filtradas}")

Campo salience : Una calificación (subjetiva) de la prominencia del sonido. 
Valores: 1 = primer plano, 2 = fondo. 
Se filtra los audios con calificación de primer plano. 

Cantidad filas afectadas por filtro: 3030


### Generación de inputs modelo

In [22]:
dataset_urbansound8k_mfcc10_df.drop(columns=['slice_file_name', 'salience'], inplace=True) 

print(f"Cantidad de columnas del dataset final: {dataset_urbansound8k_mfcc10_df.shape[1]}")
print(f"Cantidad de filas del dataset final: {dataset_urbansound8k_mfcc10_df.shape[0]}")
dataset_urbansound8k_mfcc10_df.to_csv(f"../../data/processed/dataset_mfcc_10.csv", sep=';', index=False)

Cantidad de columnas del dataset final: 112
Cantidad de filas del dataset final: 8732


In [23]:
dataset_urbansound8k_mfcc13_df.drop(columns=['slice_file_name', 'salience'], inplace=True) 

print(f"Cantidad de columnas del dataset final: {dataset_urbansound8k_mfcc13_df.shape[1]}")
print(f"Cantidad de filas del dataset final: {dataset_urbansound8k_mfcc13_df.shape[0]}")
dataset_urbansound8k_mfcc13_df.to_csv(f"../../data/processed/dataset_mfcc_13.csv", sep=';', index=False)

Cantidad de columnas del dataset final: 145
Cantidad de filas del dataset final: 8732


In [24]:
dataset_urbansound8k_mfcc15_df.drop(columns=['slice_file_name', 'salience'], inplace=True) 

print(f"Cantidad de columnas del dataset final: {dataset_urbansound8k_mfcc15_df.shape[1]}")
print(f"Cantidad de filas del dataset final: {dataset_urbansound8k_mfcc15_df.shape[0]}")
dataset_urbansound8k_mfcc15_df.to_csv(f"../../data/processed/dataset_mfcc_15.csv", sep=';', index=False)

Cantidad de columnas del dataset final: 167
Cantidad de filas del dataset final: 8732


In [25]:
dataset_urbansound8k_mfcc20_df.drop(columns=['slice_file_name', 'salience'], inplace=True) 

print(f"Cantidad de columnas del dataset final: {dataset_urbansound8k_mfcc20_df.shape[1]}")
print(f"Cantidad de filas del dataset final: {dataset_urbansound8k_mfcc20_df.shape[0]}")
dataset_urbansound8k_mfcc20_df.to_csv(f"../../data/processed/dataset_mfcc_20.csv", sep=';', index=False)

Cantidad de columnas del dataset final: 222
Cantidad de filas del dataset final: 8732
